# RNN 的致命问题

## (1)vanilla RNN公式
$$\begin{array}{c}
h_{t}=\tanh \left(W_{h} h_{t-1}+W_{x} x_{t}+b\right) \\
y_{t}=W_{y} h_{t}
\end{array}$$

随着输入序列的增大，h_t中包含较前的信息就越来越少。信息会随着时间步的增多而逐渐丢失，无法补捉长距离依赖，而有的语句恰恰是距离很远的地方起到了关键作用。

RNN 的长依赖失败不是因为输入梯度有问题，而是因为隐藏状态在时间维度上的梯度连乘导致早期状态对损失几乎没有贡献，所以模型学不会‘早期信息很重要’。所以这里我们重点讨论h_t

## (2)反向传播部分
我们关心的是：
>损失L_t对早期隐状态h1的梯度：
$$\frac{\partial L_{T}}{\partial h_{1}}$$

链式法则展开：
$$\frac{\partial L_{T}}{\partial h_{1}}=\frac{\partial L_{T}}{\partial h_{T}} \cdot \frac{\partial h_{T}}{\partial h_{T-1}} \cdot \frac{\partial h_{T-1}}{\partial h_{T-2}} \cdots \frac{\partial h_{2}}{\partial h_{1}}$$
$$\frac{\partial L_{T}}{\partial h_{1}}=\frac{\partial L_{T}}{\partial h_{T}} \prod_{t=2}^{T} \frac{\partial h_{t}}{\partial h_{t-1}}$$

回到RNN的定义：
$$h_{t}=\tanh \left(W_{h} h_{t-1}+W_{x} x_{t}+b\right)$$

对h_{t-1}求导：
$$\frac{\partial h_{t}}{\partial h_{t-1}}=\frac{\partial \tanh \left(a_{t}\right)}{\partial a_{t}} \cdot \frac{\partial a_{t}}{\partial h_{t-1}}$$

其中：
$$a_{t}=\left(W_{h} h_{t-1}+W_{x} x_{t}+b\right)$$

- 激活函数的导数：
$$\tanh ^{\prime}(x)=1-\tanh ^{2}(x)$$

- 线性部分的导数：
$$\frac{\partial a_{t}}{\partial h_{t-1}}=W_{h}$$

合在一起（Jacobian）：
$$\frac{\partial h_{t}}{\partial h_{t-1}}=W_{h}^{\top} \cdot \operatorname{diag}\left(\tanh ^{\prime}\left(a_{t}\right)\right)$$

再把结果带回去：
$$\prod_{t=2}^{T} \frac{\partial h_{t}}{\partial h_{t-1}}=\prod_{t=2}^{T}\left(W_{h}^{\top} \cdot \operatorname{diag}\left(\tanh ^{\prime}\left(a_{t}\right)\right)\right)$$

>设:
$$\begin{array}{l}
\left\|W_{h}\right\| \approx \lambda \\
\tanh ^{\prime}\left(a_{t}\right) \approx \gamma, \text { 且 } \gamma<1
\end{array}$$

那么：
$$\left\|\prod_{t=2}^{T} \frac{\partial h_{t}}{\partial h_{t-1}}\right\| \approx(\lambda \cdot \gamma)^{T}$$

这样就会出现下面的结局：
| 情况                   | 数学结果   | 现象       |
| -------------------- | ------ | -------- |
| ($\lambda \gamma$ < 1) | 指数 → 0 | **梯度消失** |
| ($\lambda \gamma$ > 1) | 指数 → ∞ | **梯度爆炸** |


## (3)上面的推导结果直接导致RNN模型无法解决长距离依赖问题

梯度消失 = 无法学习远处信息

如果：
$$\frac{\partial L_{T}}{\partial h_{1}} \approx 0$$

那么：
- h₁ 对损失几乎没影响

- 网络 学不到“早期信息是有用的”

- 参数更新只看最近几个 token

这就是RNN“记不住”的本质

## (4)RNN的另一个问题：模型不能进行并行计算，RNN必须按顺序处理，每个时间步必须依赖上一个时间步的隐藏状态的计算结果

## (5)RNN原理图
### 1.时间展开

In [ ]:
x1     x2     x3     ...     xT
 |      |      |              |
 v      v      v              v
[h1] -> [h2] -> [h3] -> ... -> [hT]
                                   |
                                   v
                                  Loss


### 2.反向传播(梯度回传路径)

In [ ]:
Loss
 |
 v
∂L/∂hT
 |
 v
∂L/∂hT-1 = ∂L/∂hT · ∂hT/∂hT-1
 |
 v
∂L/∂hT-2 = ∂L/∂hT-1 · ∂hT-1/∂hT-2
 |
 v
...
 |
 v
∂L/∂h1 = ∂L/∂hT · Π Jacobian
